In [218]:
from openpyxl import Workbook
import pandas as pd
import re
import xlwings

In [219]:
columna = ['cod_alumno','ape_paterno','ape_materno','nom_alumno','tel_alumno','movil_alumno','coe_alumno','coe_alu_personal','tipo_doc','doc_alumno','año_ingreso','periodo_egreso','escuela']

In [220]:
conta = pd.read_excel("./EGRESADOS/BASE DE DATOS CONTABILIDAD EGRESADOS.xlsx",sheet_name="Hoja1")
gestion = pd.read_excel("./EGRESADOS/BASE DE DATOS EGRESADOS GESTIÓN PÚBLICA.xlsx",sheet_name="Hoja1")
audit = pd.read_excel("./EGRESADOS/BASE DE DATOS AUDITORIA EGRESADOS.xlsx",sheet_name="Hoja1")

In [221]:
conta['escuela']='contabilidad'
gestion['escuela']='gestión pública'
audit['escuela']='auditoría'

In [222]:
db = pd.concat([conta,gestion,audit]).reset_index(drop=True)

In [223]:
for col in db.columns:
    if col == "anio_ingreso":
        db.rename(columns={'anio_ingreso':'año_ingreso'},inplace=True)
    if col == 'did_alumno':
        db.rename(columns={'did_alumno':'doc_alumno'},inplace=True)
    if col == 'tel_alu_movil':
        db.rename(columns={'tel_alu_movil':'movil_alumno'},inplace=True)
    if col == 'des_doc_identidad':
        db.rename(columns={'des_doc_identidad':'tipo_doc'},inplace=True)

In [224]:
for col in db.columns:
    if col not in columna:
        db.drop(col,inplace=True,axis=1)

In [225]:
db = db.drop_duplicates().reset_index(drop=True)

In [226]:
for col in db.columns:
    db[col]=db[col].astype('string',copy=True,errors='raise')

In [227]:
db.periodo_egreso.replace('----',None,inplace=True)
db.periodo_egreso.replace('00000',None,inplace=True)
db.tel_alumno.replace('-',None,inplace=True)
db.tel_alumno.replace('0',None,inplace=True)
db.año_ingreso.replace('0',None,inplace=True)


In [228]:
db.fillna('sin_registro',inplace=True)

In [229]:
def reemplazo_tel(val):
    if val[3]=='-':
        val=val[:3]+val[4:]
    if re.search('01',val):
        val=val[2:]
    if len(val)!=7 or val[0]=='0':
        val='sin_registro'
    if not re.search('^\d+',val) and val!='sin_registro':
        val='sin_registro'
    return val
def reemplazo_mov(val):
    if val[5]=='-':
        val=val[:5]+val[6:]
    if len(val)!=9 or val[0]!= '9':
        val='sin_registro'
    return val
def reemplazo_año_ingreso(val):
    if len(val)!=4:
        val='sin_registro'
    return val
def reemplazo_periodo_egreso(val):
    if val!='sin_registro' and val[4]!='-':
        val=val[:4]+'-'+val[-1]
    return val
def correct_doc_alumno(val):
    if len(val)!=8 and len(val)!=10:
        val="sin_registro"
    if len(val)==10 and val[8]=='.':
        val=val[:8]
    return val

In [230]:
db.tel_alumno = db.tel_alumno.apply(lambda x: reemplazo_tel(x))
db.movil_alumno = db.movil_alumno.apply(lambda x: reemplazo_mov(x))
db.año_ingreso = db.año_ingreso.apply(lambda x: reemplazo_año_ingreso(x))
db.periodo_egreso = db.periodo_egreso.apply(lambda x: reemplazo_periodo_egreso(x))
db.doc_alumno = db.doc_alumno.apply(lambda x: correct_doc_alumno(x))

In [232]:
db2 = db[db.doc_alumno!="sin_registro"]
db2


,cod_alumno,año_ingreso,ape_paterno,ape_materno,nom_alumno,tel_alumno,coe_alumno,periodo_egreso,doc_alumno,coe_alu_personal,movil_alumno,tipo_doc,escuela
0,10110156,2010,DÁVILA,LÓPEZ,ANITA,sin_registro,10110156@unmsm.edu.pe,sin_registro,48325608,a.davila22@hotmail.com,953770633,Doc. Nacional de Identidad,contabilidad
1,10110253,2010,VEGA,QUISPE,MARILUZ ALEXANDRA,5724202,10110253@unmsm.edu.pe,sin_registro,70902943,mariluz_alexa_13@hotmail.com,938957173,Doc. Nacional de Identidad,contabilidad
2,11110165,2011,MENDOZA,FELIPE,JHON ANDERSON,sin_registro,11110165@unmsm.edu.pe,sin_registro,48465886,johnandermf@gmail.com,932549794,Doc. Nacional de Identidad,contabilidad
3,13110027,2013,FUERTES,MATEO,GIANCARLO JORDY DAVID,6068170,giancarlo.fuertes@unmsm.edu.pe,sin_registro,73808591,gjordyfm@gmail.com,923764758,Doc. Nacional de Identidad,contabilidad
4,14110062,2014,VILLACORTA,TISOC,CAROL YHOSELINNE,sin_registro,carol.villacorta@unmsm.edu.pe,sin_registro,48429448,carolvillacortat@gmail.com,994968868,Doc. Nacional de Identidad,contabilidad
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11349,11110099,2011,GUARDAMINO,OJEDA,MARCO ANTONIO,sin_registro,11110099@unmsm.edu.pe,sin_registro,71951870,sin_registro,sin_registro,Doc. Nacional de Identidad,auditoría
11350,11110100,2011,HUAMANI,JUAREZ,YERZON,sin_registro,11110100@unmsm.edu.pe,sin_registro,45800924,yerzo_2008@hotmail.com,sin_registro,Doc. Nacional de Identidad,auditoría
11351,11110299,2011,ANAMPA,ANCCO,MILEYNE,sin_registro,11110299@unmsm.edu.pe,sin_registro,48025405,sin_registro,sin_registro,Doc. Nacional de Identidad,auditoría
11354,10110375,2010,CORDOVA,LUDEÑA,XIMENA ALEJANDRA,sin_registro,10110375@unmsm.edu.pe,sin_registro,73434917,sin_registro,sin_registro,Doc. Nacional de Identidad,auditoría


In [190]:
egresados = db2[columna]
egresados.columns = egresados.columns.str.upper()
egresados = egresados.style.set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]).set_properties(**{'text-align': 'center'})

In [191]:
egresados.to_excel('EGRESADOS.xlsx', sheet_name = 'EGRESADOS', index=False)
with xlwings.App(visible=True) as app:
    wb = xlwings.Book('EGRESADOS.xlsx')
    for sheet in range(len(wb.sheets)):
        ws = wb.sheets[sheet]
        ws.autofit()
    wb.save()
    wb.close()